<a href="https://colab.research.google.com/github/HuaichenOvO/ML/blob/main/CSC3180_prepreocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

致力于使用机器学习来为房价预测提供公正的、透明的模型。与中介无关，本模型用以帮助普通人。

本PJ的亮点在于它
技术上实现了xxx
结论上发现了xxx

<!-- bayesian thinking:
如果房价受A因素的波动显著地高于其他房子的平均水平，那么可以认为此时期内，A相关的 -->

In [76]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [77]:
import hashlib
import os
import tarfile
import zipfile
import requests



DATA_HUB = dict()
DATA_URL = 'http://d2l-data.s3-accelerate.amazonaws.com/'

def download(name, cache_dir=os.path.join('..', 'data')): 
    """下载一个DATA_HUB中的文件，返回本地文件名。"""
    assert name in DATA_HUB, f"{name} 不存在于 {DATA_HUB}."
    url, sha1_hash = DATA_HUB[name]
    os.makedirs(cache_dir, exist_ok=True)
    fname = os.path.join(cache_dir, url.split('/')[-1])
    if os.path.exists(fname):
        sha1 = hashlib.sha1()
        with open(fname, 'rb') as f:
            while True:
                data = f.read(1048576)
                if not data:
                    break
                sha1.update(data)
        if sha1.hexdigest() == sha1_hash:
            return fname  # Hit cache
    print(f'正在从{url}下载{fname}...')
    r = requests.get(url, stream=True, verify=True)
    with open(fname, 'wb') as f:
        f.write(r.content)
    return fname

def download_extract(name, folder=None):
    fname = download(name)
    base_dir = os.path.dirname(fname)
    data_dir, ext = os.path.splitext(fname)
    if ext == '.zip':
        fp = zipfile.ZipFile(fname, 'r')
    elif ext in ('.tar', '.gz'):
        fp = tarfile.open(fname, 'r')
    else:
        assert False, '只有zip/tar文件可以被解压缩。'
    fp.extractall(base_dir)
    return os.path.join(base_dir, folder) if folder else data_dir

def download_all():
    """下载DATA_HUB中的所有文件。"""
    for name in DATA_HUB:
        download(name)

In [78]:
# !pip install d2l
# !pip install matplotlib==3.0.0
# !pip install --upgrade mxnet
# !pip install autogluon
# !pip install torch

In [215]:
%matplotlib inline
import numpy as np
import pandas as pd
import torch
from torch import nn
from d2l import torch as d2l

DATA_HUB['kaggle_house_train'] = (  
    DATA_URL + 'kaggle_house_pred_train.csv',
    '585e9cc93e70b39160e7921475f9bcd7d31219ce')

DATA_HUB['kaggle_house_test'] = (  
    DATA_URL + 'kaggle_house_pred_test.csv',
    'fa19780a7b011d9b009e8bff8e99922a8ee2eb90')

train_data = pd.read_csv(download('kaggle_house_train'))
test_data = pd.read_csv(download('kaggle_house_test'))

In [ ]:
"""
数据可视化 了解大概印象
"""

# train_data.info()
# train_data.head(2)
# train_data['SalePrice']

# 热区图
corrmat = train_data.corr()
plt.subplots(figsize = (13,10))
sns.heatmap(corrmat)

In [214]:
# """
# 预处理
# 获得 panda dataframe 格式的训练集和测试集
# """
# from sklearn.model_selection import train_test_split

# train = train_data
# test = test_data

# numeric_attri = train.dtypes[train.dtypes != 'object'].index

# # 将非numeric变量进行onehot处理
# train = pd.get_dummies(train, dummy_na=True)

# # log 房价的值 以减小训练难度
# train['SalePrice'] = np.log(train['SalePrice'])

# # 将数据normalize成0~1，随后空白数据即可填上0
# train[numeric_attri] = train[numeric_attri].apply(
#     lambda x: (x-x.mean())/(x.std()))
# train[numeric_attri] = train[numeric_attri].fillna(0).astype(float)

# # split train_x and train_y
# x = train.drop(['Id','SalePrice'], axis = 1)
# y = train[['SalePrice']]

# # there is no null data left
# # print(y.isnull().sum().max())

# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state= 42)

# print(y_train.head(10))
# print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

In [223]:
"""
数据insight
"""
# train_data.describe().T
# train_data.describe(include='object').T

corr = train_data.corr()
# plt.figure(figsize=(20,20))
# sns.heatmap(corr,annot=True)

corr_arr=corr["SalePrice"].sort_values(ascending=False)
corr_arr=pd.DataFrame(corr_arr)
corr_arr.head(11)



,SalePrice
SalePrice,1.000000
OverallQual,0.790982
GrLivArea,0.708624
GarageCars,0.640409
GarageArea,0.623431
TotalBsmtSF,0.613581
1stFlrSF,0.605852
FullBath,0.560664
TotRmsAbvGrd,0.533723
YearBuilt,0.522897


In [ ]:
"""
数据清洗：去除无关attribute
"""

# train_data.drop(columns=['Id'],axis=1,inplace=True)
# train_data.drop(columns=['Id'],axis=1,inplace=True)

In [240]:
"""
预处理2
1. 将已经是数值化的数据normalize
2. fill object value by None without null value.
"""
from copy import deepcopy
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

train = deepcopy(train_data)
test = deepcopy(test_data)

def myPreprocessing(data_set):
    # 1. 标准化数值类型的数据
    # after normalisation, the mean disappear, 
    # so we can set the null space to 0.
    numeric_features = data_set.dtypes[data_set.dtypes != 'object'].index
    data_set[numeric_features] = data_set[numeric_features].apply(
        lambda x: (x - x.mean()) / (x.std()))
    data_set[numeric_features] = data_set[numeric_features].fillna(0)

    # 2. 将非数值类型的数据encode
    object_cols=data_set.select_dtypes(include='object')
    le = LabelEncoder()
    for i in object_cols:
        data_set[i]=le.fit_transform(data_set[i])

    data_set.astype(float)


# split train_x and train_y
myPreprocessing(train)
x = train.drop(['Id','SalePrice'], axis = 1)
y = train[['SalePrice']]

# there is no null data left
print(train.shape)
# print(y.isnull().sum().max())

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state= 242)

print(train.head(5))
# print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(1460, 81)
         Id  MSSubClass  MSZoning  LotFrontage   LotArea  Street  Alley  \
0 -1.730272    0.073350         3    -0.207948 -0.207071       1      2   
1 -1.727900   -0.872264         3     0.409724 -0.091855       1      2   
2 -1.725528    0.073350         3    -0.084413  0.073455       1      2   
3 -1.723156    0.309753         3    -0.413838 -0.096864       1      2   
4 -1.720785    0.073350         3     0.574436  0.375020       1      2   

   LotShape  LandContour  Utilities  ...  PoolArea  PoolQC  Fence  \
0         3            3          0  ... -0.068668       3      4   
1         3            3          0  ... -0.068668       3      4   
2         0            3          0  ... -0.068668       3      4   
3         0            3          0  ... -0.068668       3      4   
4         0            3          0  ... -0.068668       3      4   

   MiscFeature   MiscVal    MoSold    YrSold  SaleType  SaleCondition  \
0            4 -0.087658 -1.598563  0.138730      

In [236]:
# 对 test 做同样的处理

myPreprocessing(test)

test.shape
# print(test.isnull().sum().max())

# print(test.head(10))

(1459, 80)

In [184]:
from sklearn.metrics import mean_squared_error 

In [241]:
#Linear regression model - Linear regression model doesn't seem to be working well

from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(x_train, y_train)
y_pred = reg.predict(x_test)
RMSE_score = mean_squared_error(y_test, y_pred, squared = False)

print(RMSE_score)
print('--'*10)
print(reg.score(x_test, y_test))

Score  = 301

0.7505892360852195
--------------------
0.39770600549634016


In [242]:
# Model 3 - Catboost Regression 
from catboost import CatBoostRegressor

catb = CatBoostRegressor(iterations=3500,verbose=1000)
catb.fit(x_train, y_train)
y_pred = catb.predict(x_test)

RMSE_score = mean_squared_error(y_test, y_pred, squared = False)
print('--'*10)
print(RMSE_score)

Learning rate set to 0.015153
0:	learn: 0.9984862	total: 43ms	remaining: 2m 30s
1000:	learn: 0.1429339	total: 14.6s	remaining: 36.5s
2000:	learn: 0.0839213	total: 23s	remaining: 17.2s
3000:	learn: 0.0559104	total: 28s	remaining: 4.66s
3499:	learn: 0.0463551	total: 30.5s	remaining: 0us
0.5210991173720642
--------------------
-601.3508009514136


In [243]:
# Model 4 - XGBoost Regression 
from xgboost import XGBRegressor

xgb_reg = XGBRegressor(n_estimators=3000, learning_rate=0.005)
xgb_reg.fit(x_train, y_train, early_stopping_rounds=5, eval_set=[(x_test, y_test)],verbose = 100)
y_pred = xgb_reg.predict(x_test)

RMSE_score = mean_squared_error(y_test, y_pred, squared = False)
print(RMSE_score)
print('--'*10)
print(catb.score(x_test, y_test))

[0]	validation_0-rmse:1.11208
[100]	validation_0-rmse:0.80026
[200]	validation_0-rmse:0.64880
[300]	validation_0-rmse:0.58278
[400]	validation_0-rmse:0.55465
[500]	validation_0-rmse:0.54732
[513]	validation_0-rmse:0.54726
0.5472279165460414
